# Master regulators as order parameters of gene expression states

## Supplementary Material (data analysis)

The original microarray expression data is available in the NCBI GEO database, accession GSE24759, https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE24759

The file `hematopoiesis_data.txt` contains this data mapped to HUGO gene symbols.

In [ ]:
from pandas import read_csv, DataFrame, Series
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.cluster.hierarchy import linkage

Helper function for hierarchical clustering

In [ ]:
def cluster2_(df):    
    """clustering of dataframe on both axes (only affects ordering of rows and columns for visualization)"""
    X = df.to_numpy(copy=True) 
    columns = np.array(df.columns) 
    rows = np.array(df.index)  
    L = get_ordering(linkage(X)) 
    for i in range(len(columns)):
      X[:,i] = X[L,i]
    rows[:] = rows[L]
    X = X.T
    L = get_ordering(linkage(X))
    for i in range(len(rows)):
      X[:,i] = X[L,i]
    columns[:] = columns[L]
    X = X.T
    return DataFrame(X, index=rows, columns=columns)

def get_ordering(Y):
    """Y is the linkage"""
    clusters = [[k] for k in range(len(Y)+1)]
    for c1, c2, d, l in Y:
        clusters.append(clusters[int(c1)] + clusters[int(c2)])
    return clusters[-1]

Helper function for matrix visualization

In [ ]:
def show_matrix_(df, root=plt, font_size=6, cmap=plt.cm.bwr, clim=[-1, 1], no_ticks=False):
    """plot 2d matrix with ordered column and row headers"""
    matplotlib.rc('font', size=font_size)
    R = np.array(df)
    rows = df.index
    columns = df.columns
    im = root.matshow(R)
    if not no_ticks:
        root.xticks(range(len(columns)), columns)
        root.yticks(range(len(rows)), rows)
    for label in im.axes.xaxis.get_ticklabels():
        label.set_rotation(90)
    im.set_cmap(cmap)
    im.set_clim(clim)    
    return im

### 1. Load log-2 expression data, compute median over sample replicates

In [ ]:
df = read_csv('hematopoiesis_data.txt', index_col=0, sep='\t')

with open('sample_annotations.txt') as f:
    cell_types = {}
    for l in f:
        s = l.rstrip('\n\r').split('\t')
        cell_types.setdefault(s[1], []).append(s[0])

with open('cell_types_short_form.txt') as f:
    short_form_map = {}
    for l in f:
        if not l.startswith('#'):
            x, y = l.rstrip('\n\r').split('\t')
            short_form_map[x] = y
            
df_med = DataFrame({short_form_map[k]: df[cell_types[k]].median(axis=1) for k in cell_types}) # slow!

Display gene expression distributions

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 10)
_ = df_med.hist(bins=30, color='b')

### 2. Map gene expression data to interval [-1, 1] using "soft" sign function centered at median

In [ ]:
df_mapped = np.tanh(df_med - df_med.median())

### 3. Compute "optimal" gene set that can be used in the Hopfield model

The idea is to find a gene set that is minimal in some sense but maximizes information about which cell category a sample belongs to. For the analysis here, the selection of genes $i$ was based on a heuristic that (a) controls the variance $\sigma^2$  of expression values $e_i$ across samples using a parameter $\lambda$ , $\sigma^2(e_i) > \lambda$, and (b) subsequently maximizes independence by imposing a constraint on the Pearsson correlation coefficients $r$ controlled by a parameter $\mu$, $\langle|r(e_i,e_j)|\rangle_{j\neq i} < \mu$, where $i$ is fixed and the average runs over all other genes.

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 10)
fig, ax = plt.subplots(6, 6)
for k, x in enumerate([0.1, 0.15, 0.2, 0.25, 0.3, 0.35]):
    for l, y in enumerate([0.25, 0.275, 0.3, 0.325, 0.35, 0.375]):
        X_ = df_mapped[df_mapped.var(axis=1) > x].T
        X_ = X_.T[abs(X_.corr()).mean() < y].T
        included_genes = sorted(set(X_.columns) - {'HOXA9', 'KLF1', 'GATA3', 'PAX5', 'SPI1'})
        X_ = X_[included_genes]
        show_matrix_(cluster2_(X_.T.corr()), root=ax[k, l], font_size=0, no_ticks=True, cmap=plt.cm.YlGnBu)
        ax[k, l].axis('off')
        if x == 0.25 and y == 0.3:
            ax[k, l].set_title('$\lambda=$%4.2f\n$\mu=$%5.3f $N=$%d' % (x, y, X_.shape[1]), fontsize=10, color='red')
        else:
            ax[k, l].set_title('$\lambda=$%4.2f\n$\mu=$%5.3f $N=$%d' % (x, y, X_.shape[1]), fontsize=10)
plt.tight_layout()

"Optimal" parameters $\lambda=0.25$, and $\mu=0.3$ are obtained by visual inspection of the (Pearson) correlation matrix after hierarchical clustering. Also exclude genes corresponding to the master regulators discussed in the paper.

In [ ]:
lbda = 0.25
mu = 0.3

DataFrame `X_` contains `cell types` $\times$ `genes` mapped expression data

In [ ]:
X_ = df_mapped[df_mapped.var(axis=1) > lbda].T # 0.25
X_ = X_.T[abs(X_.corr()).mean() < mu].T # 0.3
included_genes = sorted(set(X_.columns) - {'HOXA9', 'KLF1', 'GATA3', 'PAX5', 'SPI1'})
X_ = X_[included_genes]

### 4. Pearson correlation matrix $\rightarrow$ Figure 2.a    
**Note:**  `I` contains sorted cell types

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
X_corr = X_.T.corr()
X_corr_C = cluster2_(X_corr)
I = X_corr_C.index
show_matrix_(X_corr_C, font_size=10, cmap=plt.cm.YlGnBu, clim=[-0.2, 1])
plt.colorbar(shrink=0.4)

Cell types corresponding to hematopoietic stem cell, T-lyphoid, B-lyphoid, myeloid, and erythroid categories (S, T, B, M, E) associated with clusters.

In [ ]:
clusters = {'B': [u'BCELLa1', u'BCELLa2', u'BCELLa3', u'BCELLa4'],
            'T': ['TCELL2', 'TCELL6', 'TCELL7', 'TCELL8', 'TCELL4', 'TCELL3'],
            'M': [u'MONO1', u'MONO2', 'GRAN2', 'GRAN3'],
            'S': [u'HSC1', u'HSC2', 'MEP', 'ERY1'],
            'E': [u'ERY3', u'ERY4', u'ERY5']}

Master regulators (MRs) and cell type categories

In [ ]:
categories = ['E', 'T', 'M', 'B', 'S']
MRs = ['KLF1', 'GATA3', 'SPI1', 'PAX5', 'HOXA9']

### 5. Construct pattern vectors $\xi_i^k$ from average gene expression in each cluster

In [ ]:
cluster_means = DataFrame({c: X_.T[clusters[c]].mean(axis=1) for c in categories}).T
patterns = np.sign(cluster_means)
patterns.astype(int)

Check orthogonality: scalar product between patterns $\langle\!\langle\xi^k\xi^l\rangle\!\rangle$

In [ ]:
orth = patterns.dot(patterns.T)/patterns.shape[1]
orth

In [ ]:
print('Range: [%f, %f]' % (orth.min().min(), (orth-DataFrame(np.identity(5), index=orth.index, columns=orth.columns)).max().max()))

Check pattern mean $\langle\!\langle\xi^k\rangle\!\rangle$

In [ ]:
bias = patterns.mean(axis=1)
bias

In [ ]:
print('Range: [%f, %f]' % (bias.min(), bias.max()))

### 6. Expression of master regulators (MRs) $\rightarrow$ Figure 2.b
MRs expression (not log-transformed) is mapped to interval [0, 1] here


In [ ]:
A = 2**df_med
A_ = ((A.T - A.min(axis=1))/(A.max(axis=1) - A.min(axis=1)))
A_ = A_[MRs].T

In [ ]:
show_matrix_(A_[I], font_size=10, cmap=plt.cm.YlGnBu, clim=[-0.2, 1])

### 7. Estimate order parameters $\rightarrow$ Figure 2.c

In [ ]:
OPs = patterns.dot(np.sign(X_).T)/patterns.shape[1]
B_ = OPs.loc[categories]

In [ ]:
show_matrix_(B_[I], font_size=10, cmap=plt.cm.YlGnBu, clim=[-0.2, 1])

### 8. Compare OPs to MRs for mixed cell types in the myeloid/erythroid branch $\rightarrow$ Figure 3.a

In [ ]:
matplotlib.rcParams['figure.figsize'] = (4, 4)

for k, cell_type in enumerate(['GRAN1', 'GMP', 'ERY2', 'CMP']):
    data = DataFrame({'MR': A_[cell_type], 'OP': B_[cell_type].rename(index=dict(zip(categories, MRs)))})
    data = data.rename(index={
        'SPI1': 'SPI1/M', 'KLF1': 'KLF1/E', 'PAX5': 'PAX5/B', 'GATA3': 'GATA3/T', 'HOXA9': 'HOXA9/S'
    })
    data = data.loc[['KLF1/E', 'GATA3/T', 'SPI1/M', 'PAX5/B', 'HOXA9/S']]
    data.plot(kind='bar', fontsize=15, rot=45)
    plt.title(cell_type, fontsize=15)

### 9. Explore the "epigenetic landscape" potential $V(\{\phi_k\})$ constructed from the computed pattern vectors

Helper function for getting pattern for given cell categories

In [ ]:
def get_patterns(*args):
    XI = []
    for mr in args:
        XI.append(np.array([float(s) for s in patterns.T[mr]]))
    return tuple(XI)

Helper function to compute stable states for given parameters $\beta_k$:       
Start minimization of $V$ from 5 "ideal" locations of the form $(0,...,0,1,0,..,0)$. For large enough $\beta_k$ 5 different minima will be found, for smaller values there will be fewer minima.

In [ ]:
from scipy.optimize import minimize

def get_stable_states(BETA):
    
    """return None if error encountered, e.g. pure state doesn't exist for supplied beta"""
    
    C = ['B', 'E', 'M', 'S', 'T']
    XI = get_patterns(*C)
    
    B = tuple(BETA[c] for c in C)
 
    def V(PHI): # potential
        return sum(0.5*p**2/b for p, b in zip(PHI, B)) - np.log(2.*np.cosh(sum(xi*p for xi, p in zip(XI, PHI)))).mean()
 
    result = {}
    for c, t in zip(C, [(1., 0, 0, 0, 0), (0, 1., 0, 0, 0), (0, 0, 1., 0, 0), (0, 0, 0, 1., 0), (0, 0, 0, 0, 1.)]):
        t = tuple(tt*BETA[cc] for tt, cc in zip(t, C))
        try:
            phi = minimize(V, t).x 
            result[c+''] = {c: p/BETA[c] for p, c in zip(phi, C)}
        except:
            return
        
    return DataFrame(result)    


#### 9.1 Compute locations of stable states as function of $\beta=\beta_k$ (case where all parameters $\beta_k$ are the same)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8, 6)
S, P = [], []
for k in range(400):
    s = k*0.01
    P.append(get_stable_states({c: s for c in categories}))
    S.append(s)

for k in range(5):
    for l in range(5):
        P_ = [p[categories[k]][categories[l]] for p in P]
        plt.scatter(S, P_, s=1)
        
plt.plot([2, 2], [-0.1, 1.1], '--')
plt.ylim([-0.1, 1.1])

It is seen that the point $\phi_k=0$ becomes unstable around $\beta\approx 0.75$. Five stable states exist at $\beta = 2$

In [ ]:
beta = 2.
BETA = {c: beta for c in categories}

Check that $\phi_k=0$ is unstable at $\beta=2$:     
Look at second-order term of expansion of the potential, $V^{(2)}(\{\phi_k\})=\frac{1}{2}\sum_{k,l}\left(\frac{1}{\beta_k}\delta_{kl}-\langle\!\langle\xi^k\xi^l\rangle\!\rangle\right)\phi_k\phi_l.$


In [ ]:
from scipy.linalg import eigh, eig
b = Series(BETA)
db = DataFrame(np.diag(b), columns=b.index, index=b.index) 
xixi = patterns.dot(patterns.T)/patterns.shape[1]
bb = DataFrame(np.outer(b, b), index=b.index, columns=b.index)
M = 0.5 * (db / bb - xixi)
w, v = eigh(M)
print('Eigenvalues of Hessian at origin:', w)

OP values at stable states (columns are the different categories, rows are the corresponding OP components)

In [ ]:
get_stable_states(BETA)

#### 9.2 Plot cell types over potential in M-S plane

In [ ]:
matplotlib.rcParams['figure.figsize'] = (5, 5)
stems = set()
c1, c2 = 'M', 'S'
Bx, By = BETA[c1], BETA[c2]
XIx, XIy = get_patterns(c1, c2) 
Gy, Gx = np.mgrid[-0.2:1.05:100j, -0.2:1.05:100j]
PHIy, PHIx = Gy * By, Gx * Bx
N = len(XIx)
V = 0.5 * (PHIx**2/Bx + PHIy**2/By) - (1./N) * sum(np.log(2*np.cosh(XIx[i]*PHIx + XIy[i]*PHIy)) for i in range(N))
CS = plt.contourf(Gx, Gy, V, levels=[V.min()*(19-k)/19. + V.max()*k/19. for k in range(20)], cmap=plt.cm.gray_r)
plt.contour(CS, linewidths=0.5, colors='black', linestyles='solid')
plt.scatter(OPs.loc[c1], OPs.loc[c2], c='white', edgecolor='blue', linewidth=0.5)
sol = get_stable_states(BETA)
plt.scatter(sol.T[c1], sol.T[c2], marker='+', c='r') 
plt.xlabel(r'$\phi_%s$' % (c1,), fontsize=20, labelpad=-13)
plt.ylabel(r'$\phi_%s$' % (c2,), fontsize=20, labelpad=-13)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0, color='black', linewidth=0.5)
plt.axhline(1, color='black', linewidth=0.5, linestyle=(0, (5, 5)))
plt.axvline(1, color='black', linewidth=0.5, linestyle=(0, (5, 5)))
plt.xticks([0, 1])
plt.yticks([0, 1])
for s, x, y in zip(OPs.T.index, OPs.T[c1], OPs.T[c2]):
    if x + y > 0.:
        if s in {'HSC1', 'GRAN2', 'CMP', 'GMP', 'ERY2', 'MEP', 'MONO1', 'GRAN1'}:
            plt.annotate(s, (x, y), fontsize=10)